In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from os import path

root_path = '/content/gdrive/Shareddrives/PTDL WEB/Bài tập + Đồ án/Đồ án'
dataset_path = path.join(root_path, 'dataset')
val_path = path.join(dataset_path, 'val_annotater')
preprocessing_path = path.join(dataset_path, "preprocessing")

!ls "$dataset_path"
!ls "$val_path"

annotated  preprocessing  raw  review_1.xlsx  val_annotater
2700-annotated_annotations.json  val-annotated_annotations.json  val-reannotated_annotations.json


#1. Kiểm tra độ đồng thuận bằng Fleiss Kappa

### Lần 1

Để thực hiện kiểm tra độ đồng thuận cho các dữ liệu gán nhãn, trước tiên nhóm tiến hành import các thư viện liên quan để sử dụng, trong đó bao gồm:
*  pandas -  thư viện được sử dụng để xử lý và phân tích dữ liệu
*  statsmodels - thư viện để đo độ đồng thuận


In [ ]:
import json

with open(path.join(val_path, 'val-annotated_annotations.json'), 'r') as f:
    datas = json.load(f)['examples']

In [ ]:
annotater2id = {}

for data in datas:
  for classification in data['classifications']:
    for annotator in classification['classified_by']:
      annotater2id[annotator['annotator'].split("@")[0]] = annotator['annotator_id']

annotater2id

{'tuanna21411': 3,
 'uyennnp21411': 1,
 'anhnhn21411': 5,
 'tamta21411': 6,
 'nguyennt21411': 4}

In [ ]:
import pandas as pd
from collections import Counter

In [ ]:
annotater_data = {k: [] for k in annotater2id.keys()}
annotater_data.update({
    'text': [],
    'cmtid': []
})

for data in datas:

  temp_dict = {i: 'None' for i in annotater_data.keys()}
  temp_dict['text'] = data['content']
  temp_dict['cmtid'] = data['metadata']['cmtid']

  for classification in data['classifications']:
    label = classification['classname']

    for annotator in classification['classified_by']:
      name = annotator['annotator'].split("@")[0]

      temp_dict[name] = label

  for k, v in temp_dict.items():
    annotater_data[k].append(v)

drop_columns = []
for k, v in annotater_data.items():
  if Counter(v).get('None', 0) >= len(datas) * 0.1:
    drop_columns.append(k)
    continue

  print(k, Counter(v))

annotater_df = pd.DataFrame.from_dict(annotater_data)
annotater_df = annotater_df.drop(drop_columns, axis = 1)
annotater_df

tuanna21411 Counter({'Positive': 112, 'Negative': 99, 'Neutral': 88, 'None': 1})
uyennnp21411 Counter({'Negative': 116, 'Positive': 100, 'Neutral': 84})
anhnhn21411 Counter({'Negative': 131, 'Positive': 121, 'Neutral': 45, 'None': 3})
tamta21411 Counter({'Negative': 132, 'Positive': 98, 'Neutral': 70})
nguyennt21411 Counter({'Neutral': 110, 'Positive': 106, 'Negative': 84})
text Counter({'đúng với mô_tả kho xấu': 1, '24 không một đôi mẹ ơi': 1, 'đúng với mô_tả đặt áo_khoác mà giao cái qq gì vậy màu_sắc saiii chất_liệu không đúng': 1, 'giao thiếu hàng': 1, 'đúng với mô_tả đúng màu_sắc nâu chất_liệu vải vải không được đẹp lắm nhưng phù_hợp giá_cả hơi rộng nhưng bé nhà mình thích nên vẫn đánh_giá cho shop năm sao': 1, 'chất_liệu mỏng màu sắc đen đúng với mô_tả yét khá là mỏng nhưng với giá này thì cũng ok': 1, 'màu sắc đen chất mỏng': 1, 'màu_sắc đúng đúng với mô_tả không chất_liệu mỏng áo bị lỗi': 1, 'mỏng và bự to rộng không mang được hàng về nhanh hơn dự_kiến a': 1, 'he was also an adv

,tuanna21411,uyennnp21411,anhnhn21411,tamta21411,nguyennt21411,text,cmtid
0,Negative,Negative,Negative,Negative,Neutral,đúng với mô_tả kho xấu,13281399038
1,Neutral,Neutral,Negative,Negative,Neutral,24 không một đôi mẹ ơi,12198892833
2,Negative,Negative,Negative,Negative,Negative,đúng với mô_tả đặt áo_khoác mà giao cái qq gì ...,13288012286
3,Neutral,Neutral,Negative,Negative,Neutral,giao thiếu hàng,11695680345
4,Positive,Neutral,Positive,Positive,Positive,đúng với mô_tả đúng màu_sắc nâu chất_liệu vải ...,13032398614
...,...,...,...,...,...,...,...
295,Positive,Positive,Positive,Positive,Positive,mình nhận hàng rồi nha cho shop năm sao luôn k...,12626247367
296,Positive,Positive,Positive,Positive,Positive,màu_sắc xanh than đúng với mô_tả đúng chất_liệ...,12967427776
297,Positive,Positive,Positive,Positive,Positive,vải tăm nha mặc được đẹp lắm luôn mặc đi chơi ...,12502489602
298,Negative,Negative,Negative,Negative,Negative,chất vải xấu như áo cũ mua về không mặc dk ln,13345935353


In [ ]:
from statsmodels.stats.inter_rater import aggregate_raters, fleiss_kappa

fleiss_matrix, categories = aggregate_raters(annotater_df.drop(['text', 'cmtid'], axis = 1))
kappa = fleiss_kappa(fleiss_matrix)
kappa

0.6819902671886527

Sau khi đặt ra các quy tắc, nhóm tiến hành cùng nhau gắn thẻ 300 bình luận đầu tiên. Độ đồng thuận ban đầu chỉ đạt 68%, ở mức độ tốt. Tuy nhiên, để mô hình huấn luyện tốt nhất, nhóm tiến hành thảo luận và gán lại nhãn của dữ liệu

In [ ]:
error_sentences = []
for index, row in annotater_df.iterrows():
  values = list(row.to_dict().values())[:len(annotater2id)]
  if len(set(values)) > 1:
    error_sentences.append(
          {'cmtid': row['cmtid'],
          'annotated': values,
           'text': row['text']}
        )

print(len(error_sentences))

error_df = {i: [] for i in error_sentences[0].keys()}
for i in error_sentences:
  for k, v in i.items():
    error_df[k].append(v)

error_df = pd.DataFrame.from_dict(error_df)
error_df

125


,cmtid,annotated,text
0,13281399038,"[Negative, Negative, Negative, Negative, Neutral]",đúng với mô_tả kho xấu
1,12198892833,"[Neutral, Neutral, Negative, Negative, Neutral]",24 không một đôi mẹ ơi
2,11695680345,"[Neutral, Neutral, Negative, Negative, Neutral]",giao thiếu hàng
3,13032398614,"[Positive, Neutral, Positive, Positive, Positive]",đúng với mô_tả đúng màu_sắc nâu chất_liệu vải ...
4,10353479733,"[Neutral, Negative, Negative, Negative, Neutral]",màu sắc đen chất mỏng
...,...,...,...
120,13003587990,"[Positive, Neutral, Positive, Neutral, Neutral]",chất_liệu ok đúng với mô_tả được màu_sắc đẹp t...
121,13255835599,"[Neutral, Negative, Negative, Negative, Neutral]",màu_sắc dbdj đúng với mô_tả nsjsj chất_liệu mỏ...
122,13227307206,"[Negative, Negative, Positive, Positive, Posit...",màu_sắc xanh chất_liệu len mềm chất khá ok 56 ...
123,12842734854,"[Positive, Positive, Neutral, Neutral, Positive]",chất_liệu bò đúng với mô_tả 80 màu_sắc hơi khá...


Nhóm xác định các bình luận có nhãn chênh lệch nhiều để tiến hành cùng nhau đánh giá lại.

### Lần 2

In [ ]:
import json

with open(path.join(val_path, 'val-reannotated_annotations.json'), 'r') as f:
    datas_2 = json.load(f)['examples']

In [ ]:
annotater_data_2 = {k: [] for k in annotater5id.keys()}
annotater_data_2.update({
    'text': [],
    'cmtid': []
})

for data in datas_2:

  temp_dict = {i: 'None' for i in annotater_data_2.keys()}
  temp_dict['text'] = data['content']
  temp_dict['cmtid'] = data['metadata']['cmtid']

  for classification in data['classifications']:
    label = classification['classname']

    for annotator in classification['classified_by']:
      name = annotator['annotator'].split("@")[0]

      temp_dict[name] = label

  for k, v in temp_dict.items():
    annotater_data_2[k].append(v)

drop_columns_2 = []
for k, v in annotater_data_2.items():
  if Counter(v).get('None', 0) >= len(datas) * 0.1:
    drop_columns_2.append(k)
    continue

  print(k, Counter(v))

annotater_df_2 = pd.DataFrame.from_dict(annotater_data_2)
annotater_df_2 = annotater_df_2.drop(drop_columns_2, axis = 1)
annotater_df_2

uyennnp21411 Counter({'Positive': 114, 'Negative': 106, 'Neutral': 80})
anhnhn21411 Counter({'Positive': 117, 'Negative': 116, 'Neutral': 67})
tamta21411 Counter({'Negative': 113, 'Positive': 110, 'Neutral': 77})
nguyennt21411 Counter({'Positive': 114, 'Negative': 107, 'Neutral': 79})
tuanna21411 Counter({'Positive': 116, 'Negative': 111, 'Neutral': 72, 'None': 1})
text Counter({'nói_chung là xấu vải áo xấu lắm': 1, 'giao bé quá thất_vọng': 1, 'màu_sắc nâu đúng với mô_tả 10 10 chất_liệu nỉ bông áo vải không dày không mỏng mình thích mặc rộng nên chọn size lớn sợ rộng nhma vừa in m6 40 không qua mông nhé': 1, 'chất_liệu đẹp màu_sắc đúng màu đúng với mô_tả ngoài đời còn đẹp hơn trên hình': 1, 'màu sắc đen chất mỏng': 1, 'áo kiểu gì mà tay dài tay ngắn không hiểu shop làm kiểu gì luôn': 1, 'tặng vớ đã đi rồi': 1, 'màu_sắc tyuio chất_liệu tyui đúng với mô_tả rewas tyuioplkjhgf': 1, 'chất_liệu mỏng như lá lúa mặc có phần khó_chịu đúng với mô_tả giao_diện đúng khoảng 80 màu_sắc đặt màu be mà

,uyennnp21411,anhnhn21411,tamta21411,nguyennt21411,tuanna21411,text,cmtid
0,Negative,Negative,Negative,Negative,Negative,nói_chung là xấu vải áo xấu lắm,12163393239
1,Negative,Negative,Negative,Negative,Negative,giao bé quá thất_vọng,12860404226
2,Neutral,Positive,Positive,Positive,Positive,màu_sắc nâu đúng với mô_tả 10 10 chất_liệu nỉ ...,13284156815
3,Positive,Positive,Positive,Positive,Positive,chất_liệu đẹp màu_sắc đúng màu đúng với mô_tả ...,11671891913
4,Neutral,Negative,Negative,Negative,Negative,màu sắc đen chất mỏng,10353479733
...,...,...,...,...,...,...,...
295,Neutral,Neutral,Neutral,Neutral,Neutral,ship thân_thiện đóng_gói cẩn_thận đc tặng thêm...,12280051239
296,Neutral,Neutral,Neutral,Neutral,Neutral,giao hàng nhanh quần rất dài so với ng m55 nha...,12352726771
297,Positive,Positive,Neutral,Positive,Positive,đúng với mô_tả <emoji> thumbs up </emoji> màu_...,11529783187
298,Positive,Positive,Positive,Positive,Positive,đúng với mô_tả yes màu_sắc vàng <emoji> happy ...,13274120293


In [ ]:
fleiss_matrix_2, categories = aggregate_raters(annotater_df_2.drop(['text', 'cmtid'], axis = 1))
kappa_2 = fleiss_kappa(fleiss_matrix_2)
kappa_2

0.8629463886881026

Sau khi gán nhãn dữ liệu lại, nhóm kiểm tra độ đồng thuận và thấy được kết quả đạt 86%, đây là mức kết quả rất tốt. Do đó nhóm tiếp tục gán 2700 comment còn lại để tiến hành huấn luyện mô hình

#2. Huấn luyện mô hình

##2.1. Chuẩn bị dữ luyện huấn luyện

In [ ]:
with open(path.join(val_path, "val-reannotated_annotations.json"), 'r') as f_300:
  data_300 = json.load(f_300)["examples"]

In [ ]:
with open(path.join(val_path, "2700-annotated_annotations.json"), "r") as f_2700:
  data_2700 = json.load(f_2700)["examples"]

In [ ]:
records = []

for cmt_data in data_300:
  class_counts = {}
  for classification in cmt_data['classifications']:
      # Tạo một dict để đếm số lượng các class
      class_name = classification['classname']
      n_annotators = len(set([i['annotator_id'] for i in classification['classified_by']]))
      class_counts[class_name] = n_annotators

  # Lấy class có số lượng lớn nhất
  most_common_class = max(class_counts, key=class_counts.get)

  # Lấy thông tin cần trích xuất
  cmtid = cmt_data['metadata']['cmtid']
  content =cmt_data['content']
  rating_star = cmt_data['metadata']['rating_star']

  # Thêm thông tin vào list records
  records.append({
      'cmtid': cmtid,
      'content': content,
      'classname': most_common_class,
      'rating_star': rating_star
  })

# Tạo DataFrame từ list records
df = pd.DataFrame(records)

# In ra DataFrame
df

,cmtid,content,classname,rating_star
0,12163393239,nói_chung là xấu vải áo xấu lắm,Negative,1
1,12860404226,giao bé quá thất_vọng,Negative,1
2,13284156815,màu_sắc nâu đúng với mô_tả 10 10 chất_liệu nỉ ...,Positive,5
3,11671891913,chất_liệu đẹp màu_sắc đúng màu đúng với mô_tả ...,Positive,5
4,10353479733,màu sắc đen chất mỏng,Negative,1
...,...,...,...,...
295,12280051239,ship thân_thiện đóng_gói cẩn_thận đc tặng thêm...,Neutral,3
296,12352726771,giao hàng nhanh quần rất dài so với ng m55 nha...,Neutral,3
297,11529783187,đúng với mô_tả <emoji> thumbs up </emoji> màu_...,Positive,3
298,13274120293,đúng với mô_tả yes màu_sắc vàng <emoji> happy ...,Positive,5


300 comment đầu tiên nhóm gán nhãn sẽ có 5 nhãn, do đó nhóm tiến hành lựa chọn nhãn có độ xuất hiện cao nhất để gán cho comment.

In [ ]:
for cmt_data in data_2700:
  # Lấy thông tin cần trích xuất
  cmtid = cmt_data['metadata']['cmtid']
  content =cmt_data['content']
  rating_star = cmt_data['metadata']['rating_star']
  class_name = cmt_data['classifications'][0]['classname']

  # Thêm thông tin vào list records
  records.append({
      'cmtid': cmtid,
      'content': content,
      'classname': class_name,
      'rating_star': rating_star
  })

# Tạo DataFrame từ list records
df = pd.DataFrame(records)

# In ra DataFrame
df

,cmtid,content,classname,rating_star
0,12163393239,nói_chung là xấu vải áo xấu lắm,Negative,1
1,12860404226,giao bé quá thất_vọng,Negative,1
2,13284156815,màu_sắc nâu đúng với mô_tả 10 10 chất_liệu nỉ ...,Positive,5
3,11671891913,chất_liệu đẹp màu_sắc đúng màu đúng với mô_tả ...,Positive,5
4,10353479733,màu sắc đen chất mỏng,Negative,1
...,...,...,...,...
2995,13090938856,đúng với mô_tả đúng với mô_tả màu_sắc be nâu c...,Neutral,5
2996,9720766380,mình nhận đc hàng rồi mua của shop lần thứ ba ...,Positive,5
2997,13304661640,mọi người nên suy_nghĩ trước khi mua nhé thật_...,Negative,5
2998,13268355664,đúng với mô_tả gửi đúng màu đúng mẫu màu_sắc m...,Positive,5


Tiến hành gộp 3000 comments đã gán nhãn lại để tạo ra bộ dữ liệu huấn luyện mô hình

In [ ]:
import csv

data_v2 = pd.read_csv(path.join(preprocessing_path, 'fashion_v2.csv'))

data_v2

,cmtid,itemid,name,comment,rating_star,clean_cmt
0,13239687733,23886482112,Áo len cardigan trơn dáng rộng AO26,mặc khá mát vải này cũng ổn đẹp tuyệt với cho ...,5,mặc khá mát vải này cũng ổn đẹp tuyệt với cho ...
1,13239626315,23886482112,Áo len cardigan trơn dáng rộng AO26,hàng giao nhanhh chất lươngg sản phẩm tuyệt vo...,5,hàng giao nhanhh chất lươngg sản_phẩm tuyệt vo...
2,13239565011,23886482112,Áo len cardigan trơn dáng rộng AO26,"Sp rất đẹp ,thơm, shop giao siêu nhanh, chất v...",5,sản_phẩm rất đẹp thơm shop giao siêu nhanh chấ...
3,13175587025,18894188708,"Áo Khoác Thom , Áo Khoác Lông Cừu 4 Sọc Cánh T...",Màu sắc:ko\nChất liệu:ko cos\nĐúng với mô tả:k...,1,màu_sắc không chất_liệu không cos đúng với mô_...
4,13253091864,18894188708,"Áo Khoác Thom , Áo Khoác Lông Cừu 4 Sọc Cánh T...",Màu sắc:đen\nĐúng với mô tả:cũng OK\nChất liệu...,1,màu sắc đen đúng với mô_tả cũng ok chất_liệu c...
...,...,...,...,...,...,...
14298,12768980268,22648063543,Quần kaki túi hộp nữ form suông rộng MIAA lưng...,"Trời ơi quần đẹp xỉu tụi bây ơi, thề mua được ...",5,trời_ơi quần đẹp xỉu tụi bây ơi thề mua được c...
14299,13249750307,22648063543,Quần kaki túi hộp nữ form suông rộng MIAA lưng...,"Quần vải đẹp, màu xinh lắm, form quần ưng nha,...",5,quần vải đẹp màu xinh lắm form quần ưng nha gi...
14300,13239621971,22386234995,"N07 Sét Nhung QC Phối Viền, Sét Bộ Ngủ Cho Nữ ...",M63 45kg mặc sz L vừa chiều dài nha. Hàng đẹp ...,5,m 63 45 kilogram mặc size làm vừa chiều dài nh...
14301,13240092618,22386234995,"N07 Sét Nhung QC Phối Viền, Sét Bộ Ngủ Cho Nữ ...","""Chất liệu như mô tả\nNhìn đẹp, sang nên mua đ...",5,chất_liệu như mô_tả nhìn đẹp sang nên mua để m...


###Thay thế comment pre-processing cũ bằng comment pre-processing mới

In [ ]:
df_new = pd.merge(df, data_v2, how = "left", on = "cmtid")
train_data = df_new[['cmtid','content','classname','clean_cmt','rating_star_x']]
train_data.rename(columns={'rating_star_x': 'rating_star'}, inplace=True)
train_data

<ipython-input-10-23d185bb16b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns={'rating_star_x': 'rating_star'}, inplace=True)


,cmtid,content,classname,clean_cmt,rating_star
0,12163393239,nói_chung là xấu vải áo xấu lắm,Negative,nói_chung là xấu vải áo xấu lắm,1
1,12860404226,giao bé quá thất_vọng,Negative,giao bé quá thất_vọng,1
2,13284156815,màu_sắc nâu đúng với mô_tả 10 10 chất_liệu nỉ ...,Positive,màu_sắc nâu đúng với mô_tả 10 10 chất_liệu nỉ ...,5
3,11671891913,chất_liệu đẹp màu_sắc đúng màu đúng với mô_tả ...,Positive,chất_liệu đẹp màu_sắc đúng màu đúng với mô_tả ...,5
4,10353479733,màu sắc đen chất mỏng,Negative,màu sắc đen chất mỏng,1
...,...,...,...,...,...
2995,13090938856,đúng với mô_tả đúng với mô_tả màu_sắc be nâu c...,Neutral,đúng với mô_tả đúng với mô_tả màu_sắc be nâu c...,5
2996,9720766380,mình nhận đc hàng rồi mua của shop lần thứ ba ...,Positive,mình nhận được hàng rồi mua của shop lần thứ b...,5
2997,13304661640,mọi người nên suy_nghĩ trước khi mua nhé thật_...,Negative,mọi người nên suy_nghĩ trước khi mua nhé thật_...,5
2998,13268355664,đúng với mô_tả gửi đúng màu đúng mẫu màu_sắc m...,Positive,đúng với mô_tả gửi đúng màu đúng mẫu màu_sắc m...,5


Trong quá trình gán nhãn, nhóm có bổ sung các từ viết tắt cũng như teencode, do đó các clean_cmt cần được cập nhật lại để đảm bảo độ chính xác

In [ ]:
train_data['content'] = train_data['clean_cmt'].apply(lambda x: x.replace("_", " "))
train_data

<ipython-input-11-5101c5f451fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['content'] = train_data['clean_cmt'].apply(lambda x: x.replace("_", " "))


,cmtid,content,classname,clean_cmt,rating_star
0,12163393239,nói chung là xấu vải áo xấu lắm,Negative,nói_chung là xấu vải áo xấu lắm,1
1,12860404226,giao bé quá thất vọng,Negative,giao bé quá thất_vọng,1
2,13284156815,màu sắc nâu đúng với mô tả 10 10 chất liệu nỉ ...,Positive,màu_sắc nâu đúng với mô_tả 10 10 chất_liệu nỉ ...,5
3,11671891913,chất liệu đẹp màu sắc đúng màu đúng với mô tả ...,Positive,chất_liệu đẹp màu_sắc đúng màu đúng với mô_tả ...,5
4,10353479733,màu sắc đen chất mỏng,Negative,màu sắc đen chất mỏng,1
...,...,...,...,...,...
2995,13090938856,đúng với mô tả đúng với mô tả màu sắc be nâu c...,Neutral,đúng với mô_tả đúng với mô_tả màu_sắc be nâu c...,5
2996,9720766380,mình nhận được hàng rồi mua của shop lần thứ b...,Positive,mình nhận được hàng rồi mua của shop lần thứ b...,5
2997,13304661640,mọi người nên suy nghĩ trước khi mua nhé thật ...,Negative,mọi người nên suy_nghĩ trước khi mua nhé thật_...,5
2998,13268355664,đúng với mô tả gửi đúng màu đúng mẫu màu sắc m...,Positive,đúng với mô_tả gửi đúng màu đúng mẫu màu_sắc m...,5


Khác với tiếng Anh, đặc trưng của ngôn ngữ tiếng Việt là từ ghép, do đó nhóm tiến hành huấn luyện mô hình trên 2 tập dữ liệu là tập dữ liệu có tách từ (word_segment) và không có tách từ để đo độ hiệu quả của mô hình.

* Cột content là cột chứa các comment không được tách từ
* Cột clean_cmt là cột chứa các comment được tách từ

##2.2. Machine Learning

Để thực hiện huấn luyện mô hình bằng Máy học, trước tiên nhóm tiến hành import các thư viện liên quan để sử dụng, trong đó bao gồm:
*   sklearn -  thư viện Machine Learning của Python, cung cấp nhiều mô hình và thuật toán Machine Learning khác nhau
*   numpy - thư viện cung cấp các hàm và cấu trúc dữ liệu để xử lý dữ liệu số
*   transformers - thư viện cung cấp các mô hình Transformer bao gồm BERT, RoBERTa, v.v.
*   more_itertools - thư viện cung cấp các hàm mở rộng cho các đối tượng iterable.
*   torch - thư viện Machine Learning dựa trên GPU của Python, cung cấp các mô hình và thuật toán Machine Learning được tối ưu hóa cho GPU.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder

###Tạo hàm chuyển văn bản thành vector

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, AutoModel
from more_itertools import chunked
import torch

class TransformerEmbedding(TransformerMixin, BaseEstimator):

    def __init__(self, model_name='bert-base-uncased', batch_size=1, layer=-1):
        self.model_name = model_name
        self.layer = layer
        self.batch_size = batch_size
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModel.from_pretrained(self.model_name).to(self.device)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        res = []
        total = 0
        for batch in chunked(X, self.batch_size):
            encoded_input = self.tokenizer(
                batch,
                max_length = 256,
                return_tensors='pt',
                padding = 'max_length',
                truncation = True,
                pad_to_max_length = True,
                add_special_tokens = True,
            ).to(self.device)

            with torch.no_grad():
              output = self.model(**encoded_input)
              embed = output.last_hidden_state[:,-1].cpu().detach().numpy()
              res.append(embed)
        return np.concatenate(res)


model_name = 'vinai/phobert-base-v2'

bert_embed = TransformerEmbedding(model_name, batch_size = 128)

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
X = bert_embed.transform(train_data['clean_cmt'])
X_no_word_segment = bert_embed.transform(train_data['content'])

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_data['classname'])

###Cài đặt K-Fold



Khi có một lượng dữ liệu đủ lớn, tập dữ liệu thường sẽ được chia theo tỉ lệ 80/20 để tiến hành huấn luyện và kiểm thử mô hình. Tuy nhiên, với bộ huấn luyện 3000 dòng dữ liệu thì việc chia như vậy có thể dẫn đến model hoạt động kém. Do đó, nhóm chọn **K-Fold Cross Validation** để lấy mẫu đánh giá mô hình.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Define k-fold cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
scoring = {'accuracy', 'f1_weighted'}

X = scaler.fit_transform(X)
X_no_word_segment = scaler.fit_transform(X_no_word_segment)

**Thông số đo lường**

*   Accuracy - độ chính xác
*   F1 score - giá trị trung bình của độ chính xác và khả năng thu hồi



###Huấn luyện mô hình Máy học Naive Bayes, Logistic Regression, và Linear Support Vector Machine



1.   Naive Bayes


> Có rất nhiều thuật toán học máy được sử dụng để giải quyết cho bài toán phân lớp văn bản (Text Classification). Tuy nhiên, Naïve Bayes là thuật toán có thời gian chạy nhanh và độ chính xác cao nên thường được sử dụng cho các bài toán phân lớp văn bản. Trong một nghiên cứu của tác giả Đặng Văn Nam (2020) trình bày cụ thể việc xây dựng một mô hình học máy với thuật toán Naïve Bayes sử dụng đặc trưng TF-IDF (Term Frequency – Inverse DocumentFrequency) trong phân lớp văn bản đã cho thấy hiệu quả của thuật toán Naive Bayes so với các thuật toán học máy khác với thời gian huấn luyện, kiểm thử mô hình rất nhanh và độ chính
xác cao.


2.   Logistic Regression


> Hồi quy Logistic (Logistic Regression) là thuật toán Supervised Learning (học máy có giám sát), là một kỹ thuật phân tích dữ liệu sử dụng toán học để tìm ra mối quan hệ giữa hai yếu tố dữ liệu. Sau đó, kỹ thuật này sử dụng mối quan hệ đã tìm được để dự đoán giá trị của một yếu tố dựa trên yếu tố còn lại. Trong nghiên cứu của nhóm tác giả Priyanshi Kathuria, Parth Sethi, Rithwick Negi (2022) về bình luận và rating trên các sàn thương mại điện tử, kết quả dự đoán cho thấy với mô hình Logistic Regression cho kết quả có độ chính xác cao nhất ở cả hai nhóm phân tích là bình luận (88,18%) và đánh giá (80,68%)


3.   Linear Support Vector Machine


> Mô hình SVM là một mô hình có thể sử dụng cho bài toán phân lớp (classification) cũng như bài toán hồi quy (regression). Đây là thuật toán
hỗ trợ phân loại rất phổ biến và hiệu quả, có thể áp dụng trong học có giám sát hoặc bán giám sát. Trong nghiên cứu của tác giả Đặng Quang Vinh (2023) đã xem xét bài toán định giá quyền chọn sử dụng công thức Black-Scholes bằng một số thuật toán máy học có giám sát (supervised machine learning). Kết quả thực nghiệm cho thấy mô hình SVM có nhiều khả năng ước lượng giá quyền chọn với độ chính xác cao so với các mô hình truyền thống khác




In [ ]:
# Initialize Naive Bayes, Logistic Regression, LinearSVC
model = [MultinomialNB(), LogisticRegression(), LinearSVC()]
table_ml = []

for i in model:
  cross_val_scores = cross_validate(i, X, y, cv=skf, scoring=scoring)
  cross_val_scores_2 = cross_validate(i, X_no_word_segment, y, cv=skf, scoring=scoring)
  print(cross_val_scores)
  table_ml.append({
      "Model Name": type(i).__name__,
      "Mean accuracy": cross_val_scores['test_accuracy'].mean(),
      "Mean f1": cross_val_scores['test_f1_weighted'].mean(),
      "Mean accuracy without word segment": cross_val_scores_2['test_accuracy'].mean(),
      "Mean f1 without word segment": cross_val_scores_2['test_f1_weighted'].mean()

})
result_df = pd.DataFrame(table_ml)
result_df.set_index("Model Name", inplace=True)

result_df

{'fit_time': array([0.02172709, 0.01031828, 0.00990748, 0.00944304, 0.0157845 ]), 'score_time': array([0.00792313, 0.00620461, 0.0067296 , 0.00369859, 0.0058043 ]), 'test_accuracy': array([0.48333333, 0.48833333, 0.48166667, 0.47      , 0.48      ]), 'test_f1_weighted': array([0.41713045, 0.41994243, 0.42025307, 0.41625397, 0.41494876])}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

{'fit_time': array([1.27673912, 1.5200038 , 1.56248832, 1.63424158, 2.2686758 ]), 'score_time': array([0.01312828, 0.00656676, 0.00926089, 0.00823379, 0.01321769]), 'test_accuracy': array([0.75      , 0.73833333, 0.735     , 0.745     , 0.74333333]), 'test_f1_weighted': array([0.74789354, 0.73253624, 0.73141872, 0.74490288, 0.73729776])}


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

{'fit_time': array([9.25089502, 7.32827353, 8.87322879, 8.60858464, 8.11272144]), 'score_time': array([0.01170349, 0.00464034, 0.00826836, 0.00480747, 0.00550103]), 'test_accuracy': array([0.73166667, 0.73166667, 0.70666667, 0.71666667, 0.725     ]), 'test_f1_weighted': array([0.73210208, 0.72509479, 0.70463256, 0.7131655 , 0.71772441])}


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,Mean accuracy,Mean f1,Mean accuracy without word segment,Mean f1 without word segment
Model Name,,,,
MultinomialNB,0.480667,0.417706,0.469333,0.434393
LogisticRegression,0.742333,0.738810,0.751333,0.747949
LinearSVC,0.722333,0.718544,0.718000,0.716193


##2.3. Deep Learning

Để thực hiện huấn luyện mô hình bằng Học sâu, trước tiên nhóm tiến hành import các thư viện liên quan để sử dụng, trong đó bao gồm:
*   tensorflow - cung cấp nhiều mô hình và thuật toán học sâu khác nhau, cũng như các công cụ để xây dựng và huấn luyện các mô hình học sâu.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, accuracy_score
import tensorflow as tf
import tensorflow_addons as tfa

###Huấn luyện mô hình Học sâu CNN

Mô hình CNN: CNN có khả năng tự động học và phát hiện các đặc trưng không gian trong dữ liệu, điều này rất hữu ích khi xử lý các vấn đề liên quan đến ngôn ngữ tự nhiên như phân tích cảm xúc trong bình luận. Đồng thời, bình luận thường được biểu diễn dưới dạng chuỗi từ và câu. CNN có thể được sử dụng để xử lý chuỗi dữ liệu thông qua việc sử dụng các tầng convolutional để nhận diện các đặc trưng trong không gian từ. Trong một nghiên cứu của nhóm tác giả Trần Quốc Khánh, Nguyễn Trọng Ân, Hoàng Gia Phú, Lưu Đức Cảnh (2022) đã ứng dụng đề xuất phương pháp mới và hiệu quả để giải quyết bài toán phát hiện ngôn ngữ xúc phạm tiếng Việt dựa trên mô hình PhoBERT-CNN, kết quả nghiên cứu cho thấy mô hình PhoBERT-CNN có thể đạt được độ chính xác lên tới 93,2% trong bài toán phát hiện ngôn ngữ xúc phạm tiếng Việt.

#### Huấn luyện mô hình với tập dữ liệu có tách từ (word_segment)

In [ ]:
accuracys = []
f1s = []
for train_index, test_index in skf.split(X, y):
    y_train_one_hot = to_categorical(y, num_classes=3)

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_train_one_hot[train_index], y_train_one_hot[test_index]

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Xây dựng mô hình CNN
    model = Sequential()
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 2, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 4, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(3, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  )

    # # Huấn luyện mô hình
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)

    # Đánh giá mô hình
    y_pred = model.predict(X_test)

    y_pred = np.argmax(y_pred, axis = 1)
    y_test = np.argmax(y_test, axis = 1)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
    print()

    accuracys.append(accuracy)
    f1s.append(f1)

print("ACC", sum(accuracys)/len(accuracys))
print("F1", sum(f1s)/len(f1s))

19/19 [==============================] - 0s 4ms/step
F1 Score: 0.6087441917707186
Accuracy: 0.6533333333333333

19/19 [==============================] - 0s 4ms/step
F1 Score: 0.5335629896849107
Accuracy: 0.61

19/19 [==============================] - 0s 6ms/step
F1 Score: 0.5750205539815464
Accuracy: 0.6016666666666667

19/19 [==============================] - 0s 3ms/step
F1 Score: 0.5991278134681975
Accuracy: 0.6166666666666667

19/19 [==============================] - 0s 4ms/step
F1 Score: 0.5819791106741686
Accuracy: 0.62

ACC 0.6203333333333333
F1 0.5796869319159084


####Huấn luyện mô hình với tập dữ liệu không có tách từ (word_segment)

In [ ]:
accuracys = []
f1s = []
for train_index, test_index in skf.split(X_no_word_segment, y):
    y_train_one_hot = to_categorical(y, num_classes=3)

    X_train, X_test = X_no_word_segment[train_index], X_no_word_segment[test_index]
    y_train, y_test = y_train_one_hot[train_index], y_train_one_hot[test_index]

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Xây dựng mô hình CNN
    model = Sequential()
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 2, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 4, activation='relu'))
    model.add(MaxPooling1D())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(3, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  )

    # # Huấn luyện mô hình
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)

    # Đánh giá mô hình
    # evaluates = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)

    y_pred = np.argmax(y_pred, axis = 1)
    y_test = np.argmax(y_test, axis = 1)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print("F1 Score:", f1)
    print("Accuracy:", accuracy)
    print()

    accuracys.append(accuracy)
    f1s.append(f1)
    # print(f'Loss: {loss}, Accuracy: {accuracy}')

    # print(loss)

print("ACC", sum(accuracys)/len(accuracys))
print("F1", sum(f1s)/len(f1s))

19/19 [==============================] - 0s 4ms/step
F1 Score: 0.5518050431615232
Accuracy: 0.6066666666666667

19/19 [==============================] - 0s 3ms/step
F1 Score: 0.6087532659481361
Accuracy: 0.635

19/19 [==============================] - 0s 4ms/step
F1 Score: 0.5281345094577569
Accuracy: 0.5883333333333334

19/19 [==============================] - 0s 3ms/step
F1 Score: 0.5899463487567707
Accuracy: 0.6333333333333333

19/19 [==============================] - 0s 4ms/step
F1 Score: 0.568117713341757
Accuracy: 0.6166666666666667

ACC 0.616
F1 0.5693513761331888


##2.4. Transformer Model

Để thực hiện huấn luyện mô hình bằng Transformer Model, trước tiên nhóm tiến hành import các thư viện liên quan để sử dụng, trong đó bao gồm:
*   tensorflow - cung cấp nhiều mô hình và thuật toán học sâu khác nhau, cũng như các công cụ để xây dựng và huấn luyện các mô hình học sâu.
*   torch -  được sử dụng để tối ưu hóa quá trình huấn luyện mô hình Transformer trên GPU
*   transformers -  được sử dụng để tối ưu hóa quá trình huấn luyện mô hình Transformer trên GPU
*   statistics - cung cấp các hàm và cấu trúc dữ liệu để thực hiện các phép tính thống kê, được sử dụng để tính toán các chỉ số đánh giá mô hình, chẳng hạn như độ chính xác, độ nhạy, độ đặc hiệu, v.v.
*   math - cung cấp các hàm và cấu trúc dữ liệu để thực hiện các phép tính toán học để thực hiện các phép tính toán học cần thiết cho quá trình huấn luyện, chẳng hạn như tính toán gradient và tối ưu hóa hàm mất mát
*   load_metric - được sử dụng để tải các chỉ số đánh giá mô hình đã được cài đặt sẵn


###Huấn luyện mô hình Transformer PhoBERT, XLM-RoBERTa
1.   PhoBERT

PhoBERT là một mô hình transformer, một loại mô hình học máy có thể học được mối quan hệ giữa các từ trong văn bản. Mô hình này có thể được sử dụng cho nhiều tác vụ xử lý ngôn ngữ tự nhiên (NLP). Trong nghiên cứu của nhóm tác giả Ngô Văn Sơn, Nguyễn Thị Minh Nghĩa, Hoàng Thị Huế, Nguyễn Hữu Liêm, Võ Viết Minh Nhật (2022) về cải tiến của PhoBERT nhằm tăng khả năng hiểu tiếng Việt của Chatbot thông tin khách sạn, mô hình PhoBERT sau khi thực hiện điều chỉnh cho kết quả tốt nhất trong việc phân loại ý định. Chỉ số đánh giá cho thấy mô hình PhoBERT hoạt động tốt hơn nhiều so với BERT và FastText.
2.   XLM-RoBERTa

XLM-RoBERTa là một mô hình ngôn ngữ mạng nơ-ron tiên tiến được phát triển bởi Facebook AI Research (FAIR). Đây là một biến thể của mô hình Roberta, được thiết kế đặc biệt để xử lý ngôn ngữ tự nhiên và chia sẻ thông tin giữa các ngôn ngữ khác nhau. Nhóm tác giả Nguyễn Ngọc Toàn, Lê Xuân Tấn, Lương Thế Dũng, Trần Nghi Phú (2022) đã dùng mô hình XLM-RoBERTa để nghiên cứu về "nhận dạng thực thể được đặt tên trong văn bản Tiếng Việt sử dụng học máy và ứng dụng trong an ninh mạng". Kết quả cho thấy mô hình XLM-RoBERTa nhận dạng rất tốt với chỉ số của các nhãn trong nghiên cứu đều đạt trên 90%.




####Giải phóng bộ nhớ GPU trong quá trình train

In [ ]:
import gc

def free_gpu():
  torch.cuda.empty_cache()
  gc.collect()

In [ ]:
# -*- coding: utf-8 -*-
import os, pickle, re, keras, sklearn, string, io, gensim, warnings, io
import random as rn
warnings.filterwarnings("ignore")

#### Chuẩn hóa dữ liệu để đưa vào mô hình

In [ ]:
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F

def get_train_test(label, word = True):

    if word:
        X = train_data['clean_cmt'].values.tolist()
    else:
        X = train_data['content'].values.tolist()

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(train_data['classname'])
    y = F.one_hot(torch.tensor(y), num_classes=3).numpy()

    X = np.array(X)
    y = np.array(y)

    test_size = 0.2

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y, test_size=test_size,
                                                        stratify = y,
                                                        random_state=seed)

    print(X_train[0])
    print(y_train[0])
    print(X_test[0])
    print(y_test[0])
    print(X_train[0])
    print(y_train[0])
    print(X_test[0])

    return X_train, X_test, y_train, y_test

#### Load model

In [ ]:
!pip install sentencepiece
!pip install transformers[torch]
!pip install datasets

In [ ]:
from torch.nn import *
from transformers import *
import torch
import random,operator
import pickle, statistics
from sklearn.metrics import *
import math
from torch.utils.data import DataLoader
from datasets import load_metric
metric = load_metric('accuracy') # Acccuracy
from transformers import logging
import warnings
warnings.filterwarnings("ignore")

#### In cấu hình mô trường

In [ ]:
from torch import cuda
device = 'cuda'if cuda.is_available() else 'cpu'
!nvidia-smi

Sat Dec  9 04:38:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    31W /  70W |    315MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Cài đặt môi trường để tránh sự random về số

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#### Các hàm bổ trợ

##### Hàm chuyển đổi label qua id

In [ ]:
def convert_lb2id(label):
  label2id = {}
  for idx, i in enumerate(label):
    label2id[i] = idx

  return label2id

def convert_id2lb(label):
  id2label = {}
  for idx, i in enumerate(label):
    id2label[idx] = i

  return id2label

##### Tạo cấu trúc bộ dữ liệu cho máy học

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__ (self, tokenizer, label_list, X_data, y_data = None):
        self.tokenizer = tokenizer
        self.X_data = X_data
        self.y_data = y_data
        self.label_list = label_list

    def __len__ (self):
        return len(self.X_data)

    def __getitem__(self, idx):
        inputs = self.tokenizer.encode_plus(
            self.X_data[idx],
            max_length=256,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )


        return_data = {
            'input_ids' : inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
        }

        if self.y_data is not None:
          return_data['labels'] = torch.FloatTensor(self.y_data[idx])


        return return_data

##### Tạo thực thể huấn luyện

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = None  # Initialize class_weights
        self.compute_class_weights()

    def compute_class_weights(self):
        # Assuming labels are integers representing class indices
        unique_classes = np.unique(np.argmax(y, axis = 1))
        class_weights = compute_class_weight('balanced', classes=unique_classes, y=np.argmax(y, axis = 1))
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

    def compute_loss(self, model, inputs, return_outputs=False):

        if "labels" in inputs:
          labels = inputs.pop("labels")

        # Forward
        outputs = model(**inputs)
        logits = outputs.logits

        #  Compute loss
        loss_function = CrossEntropyLoss(weight = self.class_weights)       # MCC
        # loss_function = BCEWithLogitsLoss()    # MLC

        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = loss_function(logits, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss

##### Tạo hàm lấy dữ liệu để huấn luyện

In [ ]:
def get_dataset(tokenizer, label_model, X_train, X_test, y_train, y_test):
  train_dataset = CustomDataset(tokenizer, label_model, X_train, y_train)
  if type_job == "train":
    test_dataset = CustomDataset(tokenizer, label_model, X_test, y_test)
  else:
    test_dataset = CustomDataset(tokenizer, label_model, X_test)


  test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
  del test_dataset

  return train_dataset, test_dataloader

##### Tạo hàm lấy model để huấn luyện

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, AutoModelForSequenceClassification, AutoModel
def get_model(name_model):
  tokenizer = AutoTokenizer.from_pretrained(name_model)
  model = AutoModelForSequenceClassification.from_pretrained(
            name_model,
            num_labels = len(label_model),
            label2id = convert_lb2id(label_model),
            id2label = convert_id2lb(label_model),
            problem_type = problem_type,
            ignore_mismatched_sizes=True
          ).to(device)

  return model, tokenizer

##### Tạo hàm áp dụng mô hình để huấn luyện

In [ ]:
def train_model(model, training_args, train_dataset):
  model.train()
  trainer = CustomTrainer(
      model = model,
      args = training_args,
      train_dataset = train_dataset,
      # eval_dataset = test_dataset,
      data_collator = lambda data : {
          'input_ids' : torch.stack([item['input_ids'] for item in data]),
          'attention_mask' : torch.stack([item['attention_mask'] for item in data]),
          'labels' : torch.stack([item['labels'] for item in data]),
      }
  )

  trainer.train()

  del trainer

  free_gpu()

##### Tạo hàm để chuyển về dạng one-hot-vector (index to num)

In [ ]:
def get_ohv(label, name_labels):
  ohv_label = []
  for i in range(len(name_labels)):
    ohv_label.append(0)

  label_list = [i.strip() for i in label.split(",")]

  for i in label_list:
    if i in name_labels:
      ohv_label[name_labels.index(i)] = 1

  return ohv_label

##### Hàm in ra kết quả

In [ ]:
def print_predict_result(model, test_dataloader):
  y_pred = []
  y_true = []

  model.eval()
  with torch.no_grad():
      for batch in test_dataloader:
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)

          output = model(input_ids, attention_mask=attention_mask)

          logits = output.logits.detach().cpu().numpy()
          logits = np.array([np.argmax(i).flatten().item() for i in logits])

          y_pred.extend(logits)

          if type_job == "train":
            labels = batch['labels'].cpu().numpy()
            labels = [np.argmax(i) for i in labels]
            y_true.extend(labels)

  y_pred = [get_ohv(label_model[i], label_model) for i in y_pred]
  y_true = [get_ohv(label_model[i], label_model) for i in y_true]

  micro_test = f1_score(y_true, y_pred, average='micro')
  macro_test = f1_score(y_true, y_pred, average='macro')
  weighted_test = f1_score(y_true, y_pred, average='weighted')
  accuracy_test = accuracy_score(y_true,y_pred)
  report = classification_report(y_true, y_pred, target_names = label_model)
  print()
  print("Accuracy: ", round(accuracy_test*100,2))
  print("F1 weighted: ", round(weighted_test*100,2))
  print("f1 macro: ", round(macro_test*100,2))
  print("f1 micro: ", round(micro_test*100,2))

  print("-"*100)
  print(report)
  print()
  print()
  print()
  print("=" * 100 + "\n")

  return accuracy_test, weighted_test

##### Chuẩn bị mô hình

In [ ]:
name_models = ["xlm-roberta-base"
            , "vinai/phobert-base-v2"]


problem_type = "single_label_classification"

category_list = ["negative", "neutral", "positive"]

label_model = category_list

max_len = 256 #Độ dài tối đa mô hình có thể nhận vào
epochs = 5 #Số lần huấn luyện
batch_size = 32
learning_rate = 1e-4 #Tốc độ học
weight_decay = 0.01 #Phần trăm bỏ để tránh overfitting

log_strategy = "steps"
log_step = 100

In [ ]:
training_args = TrainingArguments(
    output_dir = '/content',
    # evaluation_strategy= 'epoch',
    learning_rate = learning_rate,
    # save_strategy = 'epoch',
    per_device_train_batch_size = batch_size,
    num_train_epochs = epochs,
    weight_decay = weight_decay,
    fp16=True,
    fp16_full_eval=True,
    jit_mode_eval=True,
    logging_strategy=log_strategy,
    logging_steps = log_step,
    seed = seed,
    # load_best_model_at_end=True,
)

type_job = "train"

#### Cài đặt KFold

In [ ]:
# Define k-fold cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

#### Huấn luyện mô hình với tập dữ liệu có tách từ (word_segment)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import torch.nn.functional as F

X_train, X_test, y_train, y_test = get_train_test(label_model, word = True)

X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

eval_data = {}
for train_index, test_index in skf.split(X, np.argmax(y, axis = 1)):
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    for name_model in name_models:
      free_gpu()
      model, tokenizer = get_model(name_model)
      train_dataset, test_dataloader = get_dataset(tokenizer, label_model, X_train_fold, X_test_fold, y_train_fold, y_test_fold)

      train_model(model, training_args, train_dataset)
      accuracy_test, weighted_test = print_predict_result(model, test_dataloader)
      print()

      if name_model not in eval_data:
          eval_data[name_model] = []

      eval_data[name_model].append([accuracy_test, weighted_test])

      model.save_pretrained(path.join(model_path, str(accuracy_test) + "_" + str(weighted_test)))

      del model

màu_sắc ghi sữa chất_liệu không biết đúng với mô_tả đúng giá hạt_dẻ mà áo lại xịn lắm khá dày_dặn form croptop siêu xinh ạ
[0 0 1]
đúng với mô_tả cũng không giống lắm chất_liệu hơi mỏng màu_sắc nâu hình_ảnh chỉ mang tính_chất nhận xu
[0 1 0]
màu_sắc ghi sữa chất_liệu không biết đúng với mô_tả đúng giá hạt_dẻ mà áo lại xịn lắm khá dày_dặn form croptop siêu xinh ạ
[0 0 1]
đúng với mô_tả cũng không giống lắm chất_liệu hơi mỏng màu_sắc nâu hình_ảnh chỉ mang tính_chất nhận xu


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/77de1f7a7e5e737aead1cd880979d4f1b3af6668/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading file sentencepiece.

Step,Training Loss
100,1.075300




Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in /content/gdrive/MyDrive/DFS/temp/Code_Crawl_Data_Shoppe/model/0.59_0.5149936476617732/config.json



Accuracy:  59.0
F1 weighted:  51.5
f1 macro:  45.82
f1 micro:  59.0
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

    negative       0.49      0.91      0.63       210
     neutral       1.00      0.01      0.01       151
    positive       0.79      0.67      0.73       239

   micro avg       0.59      0.59      0.59       600
   macro avg       0.76      0.53      0.46       600
weighted avg       0.74      0.59      0.51       600
 samples avg       0.59      0.59      0.59       600








...

Model weights saved in /content/gdrive/MyDrive/DFS/temp/Code_Crawl_Data_Shoppe/model/0.7766666666666666_0.7780473867969921/pytorch_model.bin
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--vinai--phobert-base-v2/snapshots/2b51e367d92093c9688112098510e6a58bab67cd/config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base-v2",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_cl

Step,Training Loss
100,0.604000




Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in /content/gdrive/MyDrive/DFS/temp/Code_Crawl_Data_Shoppe/model/0.7983333333333333_0.8013730862328813/config.json



Accuracy:  79.83
F1 weighted:  80.14
f1 macro:  78.17
f1 micro:  79.83
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

    negative       0.84      0.80      0.82       211
     neutral       0.60      0.67      0.64       150
    positive       0.90      0.87      0.89       239

   micro avg       0.80      0.80      0.80       600
   macro avg       0.78      0.78      0.78       600
weighted avg       0.81      0.80      0.80       600
 samples avg       0.80      0.80      0.80       600








Model weights saved in /content/gdrive/MyDrive/DFS/temp/Code_Crawl_Data_Shoppe/model/0.7983333333333333_0.8013730862328813/pytorch_model.bin


#### Kết quả từng fold của 2 mô hình có tách từ (word_segment)

In [ ]:
eval_data

{'xlm-roberta-base': [[0.59, 0.5149936476617732],
  [0.78, 0.7832663001150312],
  [0.7266666666666667, 0.7268739044617716],
  [0.3516666666666667, 0.18298808055898072],
  [0.7766666666666666, 0.7780473867969921]],
 'vinai/phobert-base-v2': [[0.815, 0.8162543877318738],
  [0.7983333333333333, 0.7973833595650964],
  [0.825, 0.8268434214683358],
  [0.835, 0.8374656534009067],
  [0.7983333333333333, 0.8013730862328813]]}

#### Điểm trung bình của 2 mô hình có tách từ (word_segment)

In [ ]:
for k, values in eval_data.items():
    print(k)
    accs = [i[0] for i in values]
    f1ss = [i[1] for i in values]

    print("ACC", sum(accs)/len(accs))
    print("F1", sum(f1ss)/len(f1ss))

xlm-roberta-base
ACC 0.6449999999999999
F1 0.5972338639189098
vinai/phobert-base-v2
ACC 0.8143333333333335
F1 0.8158639816798188


#### Huấn luyện mô hình với tập dữ liệu không có tách từ (word_segment)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import torch.nn.functional as F

# Define k-fold cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
X_train, X_test, y_train, y_test = get_train_test(label_model, word = False)

X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

eval_data_2 = {}
for train_index, test_index in skf.split(X, np.argmax(y, axis = 1)):
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    for name_model in name_models:
        free_gpu()
        model, tokenizer = get_model(name_model)
        train_dataset, test_dataloader = get_dataset(tokenizer, label_model, X_train_fold, X_test_fold, y_train_fold, y_test_fold)

        train_model(model, training_args, train_dataset)
        accuracy_test, weighted_test = print_predict_result(model, test_dataloader)
        print()

        if name_model not in eval_data_2:
            eval_data_2[name_model] = []

        eval_data_2[name_model].append([accuracy_test, weighted_test])

        del model

eval_data_2

màu sắc ghi sữa chất liệu không biết đúng với mô tả đúng giá hạt dẻ mà áo lại xịn lắm khá dày dặn form croptop siêu xinh ạ
[0 0 1]
đúng với mô tả cũng không giống lắm chất liệu hơi mỏng màu sắc nâu hình ảnh chỉ mang tính chất nhận xu
[0 1 0]
màu sắc ghi sữa chất liệu không biết đúng với mô tả đúng giá hạt dẻ mà áo lại xịn lắm khá dày dặn form croptop siêu xinh ạ
[0 0 1]
đúng với mô tả cũng không giống lắm chất liệu hơi mỏng màu sắc nâu hình ảnh chỉ mang tính chất nhận xu


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/77de1f7a7e5e737aead1cd880979d4f1b3af6668/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading file sentencepiece.

Step,Training Loss
100,1.107300
200,1.107200
300,1.104300




Training completed. Do not forget to share your model on huggingface.co/models =)





Accuracy:  35.0
F1 weighted:  18.15
f1 macro:  17.28
f1 micro:  35.0
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

    negative       0.35      1.00      0.52       210
     neutral       0.00      0.00      0.00       151
    positive       0.00      0.00      0.00       239

   micro avg       0.35      0.35      0.35       600
   macro avg       0.12      0.33      0.17       600
weighted avg       0.12      0.35      0.18       600
 samples avg       0.35      0.35      0.35       600








...

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--vinai--phobert-base-v2/snapshots/2b51e367d92093c9688112098510e6a58bab67cd/config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/phobert-base-v2",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "PhobertTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.35.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "voc

Step,Training Loss
100,0.689700
200,0.400800
300,0.257500




Training completed. Do not forget to share your model on huggingface.co/models =)





Accuracy:  77.83
F1 weighted:  78.09
f1 macro:  75.91
f1 micro:  77.83
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

    negative       0.82      0.78      0.80       211
     neutral       0.57      0.63      0.60       150
    positive       0.89      0.87      0.88       239

   micro avg       0.78      0.78      0.78       600
   macro avg       0.76      0.76      0.76       600
weighted avg       0.78      0.78      0.78       600
 samples avg       0.78      0.78      0.78       600








{'xlm-roberta-base': [[0.35, 0.18148148148148147],
  [0.7266666666666667, 0.7253342110363576],
  [0.765, 0.7673454134902766],
  [0.3983333333333333, 0.22694080254270954],
  [0.75, 0.7427556998372237]],
 'vinai/phobert-base-v2': [[0.8316666666666667, 0.8330371679675747],
  [0.7816666666666666, 0.7845078243456529],
  [0.8183333333333334, 0.8191500746516925],
  [0.83, 0.8323130431339713],
  [0.7783333333333333, 0.7809484097133127]]}

#### Kết quả từng fold của 2 mô hình không có tách từ (word_segment)

In [ ]:
eval_data_2

{'xlm-roberta-base': [[0.35, 0.18148148148148147],
  [0.7266666666666667, 0.7253342110363576],
  [0.765, 0.7673454134902766],
  [0.3983333333333333, 0.22694080254270954],
  [0.75, 0.7427556998372237]],
 'vinai/phobert-base-v2': [[0.8316666666666667, 0.8330371679675747],
  [0.7816666666666666, 0.7845078243456529],
  [0.8183333333333334, 0.8191500746516925],
  [0.83, 0.8323130431339713],
  [0.7783333333333333, 0.7809484097133127]]}

#### Điểm trung bình của 2 mô hình không có tách từ (word_segment)

In [ ]:
for k, values in eval_data_2.items():
    print(k)
    accs = [i[0] for i in values]
    f1ss = [i[1] for i in values]

    print("ACC", sum(accs)/len(accs))
    print("F1", sum(f1ss)/len(f1ss))

xlm-roberta-base
ACC 0.5980000000000001
F1 0.5287715216776098
vinai/phobert-base-v2
ACC 0.808
F1 0.8099913039624408


#3. Tổng kết

In [ ]:
import pandas as pd

In [ ]:
result = {'Model name': ['Naive Bayes','Logistic Regession', 'Support vector machine', 'CNN', 'XLM-R', 'Bert (PhoBert)'],
          'Accuracy': [0.480667,0.742333, 0.722333, 0.620333, 0.644999, 0.814333],
          'F1': [0.417706, 0.738810, 0.718544, 0.579687, 0.597234, 0.815864],
          'Accuracy without word segment': [0.469333, 0.751333, 0.718000, 0.616, 0.598, 0.808],
          'F1 without word segment': [0.434393, 0.747949, 0.716193,  0.569351, 0.528772, 0.809991]}
result_df = pd.DataFrame(data = result)
result_df

,Model name,Accuracy,F1,Accuracy without word segment,F1 without word segment
0,Naive Bayes,0.480667,0.417706,0.469333,0.434393
1,Logistic Regession,0.742333,0.738810,0.751333,0.747949
2,Support vector machine,0.722333,0.718544,0.718000,0.716193
3,CNN,0.620333,0.579687,0.616000,0.569351
4,XLM-R,0.644999,0.597234,0.598000,0.528772
5,Bert (PhoBert),0.814333,0.815864,0.808000,0.809991


**Đánh giá và lựa chọn mô hình**

Thống kê thấy được mô hình PhoBert và tách từ đem lại giá trị tốt nhất, do vậy nhóm lấy mô hình PhoBert có hệ số điểm F1, Accuracy cao nhất trong 5 fold là 0.835, 0.8374656534009067 để thực hiện dự đoán bộ dữ liệu có sẵn